<div class="alert alert-info">
    <h1>Analysis of Saturn's F Ring During the Cassini Mission</h1>
    <h1>Analyze Sensitivity of EW to Changes in Reprojection Parameters</h1>
    
Robert S. French, rfrench@seti.org - Last updated April 9, 2024

Research performed under NASA Grant 80NSSC21K0527 for CDAP proposal "The Recent History of Saturn's Dusty Rings".
</div>

In [5]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
if '..' not in sys.path: sys.path.append('..'); sys.path.append('../external')

from f_ring_util.f_ring import (add_hover,
                                read_ew_stats)

%matplotlib widget
mpl.rcParams['figure.max_open_warning'] = 100

# Compare EWs Using Different Radial and Longitude Resolutions and Zooms

In [7]:
obsdata_by_res = {}
for rzoom in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    for lzoom in [1, 5]:
        for rres in [1, 5]:
            for lres in [0.02, 0.10]:
                fn = f'../data_files/sensitivity/sens_ew_{rres}_{lres:4.2f}_{rzoom}_{lzoom}_0_0.csv'
                try:
                    obsdata_by_res[(rres, lres, rzoom, lzoom)] = read_ew_stats(fn, verbose=False)
                except FileNotFoundError:
                    print('Warning - Not found:', fn)
                except:
                    print('Other error:', fn)

In [24]:
agg_obsdata = pd.DataFrame()
for vers, res_obsdata in obsdata_by_res.items():
    if len(res_obsdata) == 0: continue
    sfx = f' {vers[0]:02d}/{vers[1]:4.2f}/{vers[2]:02d}/{vers[3]:02d}'
    agg_obsdata = agg_obsdata.join(res_obsdata[['Normal EW Mean']], how='outer')
    agg_obsdata.rename(columns={'Normal EW Mean': f'Normal EW Mean{sfx}'}, inplace=True)
agg_obsdata = agg_obsdata.transpose()
agg_obsdata.rename(columns={'SENS_ISS_173RF_HIRESFRNG001_PRIME': '173RF_HIRESFRNG001',
                            'SENS_ISS_181RF_FMOVIE001_PRIME': '181RF_FMOVIE001',
                            'SENS_ISS_181RF_FRINGPHOT001_VIMS': '181RF_FRINGPHOT001',
                            'SENS_ISS_196RF_FMOVIE006_PRIME': '196RF_FMOVIE006',
                            'SENS_ISS_233RI_HIRESAFRG001_PRIME': '233RI_HIRESAFRG001'},
                   inplace=True)
# We sort by 181RF_FRINGPHOT001 because it has the highest variation
agg_obsdata.sort_values(by=['181RF_FRINGPHOT001'], inplace=True)
with pd.option_context('display.width', 1000):
    print(pd.concat([agg_obsdata.describe().T,
                     (agg_obsdata.std() / agg_obsdata.mean()).rename('norm std'),
                     ((agg_obsdata.max()-agg_obsdata.min())/agg_obsdata.min()).rename('frac range')], axis=1).T)
print()
print('Radial res / Longitude res / Radial zoom / Longitude zoom')
print(agg_obsdata.to_string())

Observation  173RF_HIRESFRNG001  181RF_FMOVIE001  181RF_FRINGPHOT001  196RF_FMOVIE006  233RI_HIRESAFRG001
count                 80.000000        80.000000           80.000000        80.000000           80.000000
mean                   1.111663         0.696264            6.801135         6.539454            0.975851
std                    0.005505         0.002380            0.144866         0.015277            0.003085
min                    1.090271         0.689080            6.687075         6.520385            0.971679
25%                    1.108179         0.695548            6.694590         6.526595            0.972487
50%                    1.113555         0.696894            6.717172         6.534521            0.976794
75%                    1.115426         0.698539            6.853252         6.555123            0.979001
max                    1.121333         0.699686            7.076198         6.564467            0.981101
norm std               0.004952         0.0034

<div class="alert alert-info">
<h2>Partial Conclusion</h2>
    
<b>All EWs for mosaics with different reprojection parameters appear approximately the same except for 181RF_FRINGPHOT001 those with a longitude resolution of 0.10 km and a longitude zoom of 05. So we will try again with those removed.</b>
</div>

In [32]:
agg_obsdata = pd.DataFrame()
for vers, res_obsdata in obsdata_by_res.items():
    if len(res_obsdata) == 0: continue
    if vers[1] == 0.1 and vers[3] == 5: continue
    sfx = f' {vers[0]:02d}/{vers[1]:4.2f}/{vers[2]:02d}/{vers[3]:02d}'
    agg_obsdata = agg_obsdata.join(res_obsdata[['Normal EW Mean']], how='outer')
    agg_obsdata.rename(columns={'Normal EW Mean': f'Normal EW Mean{sfx}'}, inplace=True)
agg_obsdata = agg_obsdata.transpose()
agg_obsdata.rename(columns={'SENS_ISS_173RF_HIRESFRNG001_PRIME': '173RF_HIRESFRNG001',
                            'SENS_ISS_181RF_FMOVIE001_PRIME': '181RF_FMOVIE001',
                            'SENS_ISS_181RF_FRINGPHOT001_VIMS': '181RF_FRINGPHOT001',
                            'SENS_ISS_196RF_FMOVIE006_PRIME': '196RF_FMOVIE006',
                            'SENS_ISS_233RI_HIRESAFRG001_PRIME': '233RI_HIRESAFRG001'},
                   inplace=True)
print()
print('Chosen parameters:')
print(agg_obsdata.loc['Normal EW Mean 05/0.02/10/01'])
print()
print('Ratio to mean:')
print(agg_obsdata.loc['Normal EW Mean 05/0.02/10/01'] / agg_obsdata.mean())
agg_obsdata.sort_values(by=['181RF_FRINGPHOT001'], inplace=True)
print()
with pd.option_context('display.width', 1000):
    print(pd.concat([agg_obsdata.describe().T,
                     (agg_obsdata.std() / agg_obsdata.mean()).rename('norm std'),
                     ((agg_obsdata.max()-agg_obsdata.min())/agg_obsdata.min()).rename('frac range')], axis=1).T)
print()
print('Radial res / Longitude res / Radial zoom / Longitude zoom')
print(agg_obsdata.to_string())



Chosen parameters:
Observation
173RF_HIRESFRNG001    1.114974
181RF_FMOVIE001       0.699075
181RF_FRINGPHOT001    6.692980
196RF_FMOVIE006       6.528155
233RI_HIRESAFRG001    0.977655
Name: Normal EW Mean 05/0.02/10/01, dtype: float64

Ratio to mean:
Observation
173RF_HIRESFRNG001    1.002727
181RF_FMOVIE001       1.004195
181RF_FRINGPHOT001    0.995763
196RF_FMOVIE006       0.999265
233RI_HIRESAFRG001    1.000629
dtype: float64

Observation  173RF_HIRESFRNG001  181RF_FMOVIE001  181RF_FRINGPHOT001  196RF_FMOVIE006  233RI_HIRESAFRG001
count                 60.000000        60.000000           60.000000        60.000000           60.000000
mean                   1.111942         0.696154            6.721456         6.532954            0.977040
std                    0.006244         0.002721            0.042978         0.011521            0.002642
min                    1.090271         0.689080            6.687075         6.520385            0.971781
25%                    1.106597  

<div class="alert alert-info">
<h2>Conclusion</h2>
    
<b>All EWs for mosaics with different reprojection parameters are consistent within 0.5 to 2.9%. Our chosen parameters agree with the mean results within 0.5%. We are confident that the choice of reprojection parameters does not make much difference, and that our choice is approximately equal to the mean. Thus we will stick with our chosen parameters.</b>
</div>